<a href="https://colab.research.google.com/github/Girisha-Malni-builds01/AI-Placement-Practice-Notebook/blob/Supervised-AI-Algorithms/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load MovieLens 100k dataset (direct from GitHub mirror)
movies = pd.read_csv("https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv", nrows=2000)
ratings = pd.read_csv("https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv", nrows=20000)

# For demo: pretend these are movies instead of books
movies = movies[['book_id','title']]
ratings = ratings[['user_id','book_id','rating']]

# Step 2: Create user-item matrix
user_item_matrix = ratings.pivot(index='user_id', columns='book_id', values='rating').fillna(0)

# Step 3: Fit KNN model (cosine similarity)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix)

# Step 4: Pick a random user
user_id = 10
distances, indices = model_knn.kneighbors([user_item_matrix.loc[user_id].values], n_neighbors=6)

# Step 5: Recommend movies liked by neighbors
neighbor_ids = user_item_matrix.index[indices.flatten()]
neighbor_ratings = ratings[ratings['user_id'].isin(neighbor_ids)]
top_recs = neighbor_ratings.groupby('book_id').rating.mean().sort_values(ascending=False).head(10)
recommendations = movies[movies['book_id'].isin(top_recs.index)]

print("Recommendations for User", user_id)
print(recommendations)


Recommendations for User 10
     book_id                                              title
1          2  Harry Potter and the Sorcerer's Stone (Harry P...
3          4                              To Kill a Mockingbird
14        15                          The Diary of a Young Girl
20        21  Harry Potter and the Order of the Phoenix (Har...
23        24  Harry Potter and the Goblet of Fire (Harry Pot...
26        27  Harry Potter and the Half-Blood Prince (Harry ...
27        28                                  Lord of the Flies
116      117  A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
118      119                                The Handmaid's Tale
390      391                                          The Lorax
